In [1]:
import data_mining as dm
from lstm_autoencoder import RecurrentAutoencoder
import train
import numpy as np
import torch
import importlib
import pandas as pd
from torch import nn

from get_paquetes import capturar_paquetes
import clean_paquetes


In [2]:
device = torch.device("cpu")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

Get package

In [23]:
num_paquetes = 100
datos_capturados = capturar_paquetes(num_paquetes)

#convertir a df
#poner nombres a las columnas times, src_ips, dst_ips, protocols, lengths
df = pd.DataFrame(datos_capturados, columns = ['Time', 'Source', 'Destination', 'Protocol', 'Length'])
#add a column of info and all values are '1'
df_info = pd.DataFrame(np.ones((len(df),1)), columns = ['Info'])
df_no= pd.DataFrame(np.ones((len(df),1)), columns = ['No.'])
df = pd.concat([df, df_info], axis=1)
df = pd.concat([df_no, df], axis=1)
print(df)
df.to_csv('paquetes.csv', index=False)
#print(df)

#limpiar los datos
train_dataset,seq_len,n_features = clean_paquetes.clean_and_get_Data('paquetes.csv')
#print(df)


    No.                Time          Source      Destination Protocol Length  \
0   1.0   1712228535.151954  52.168.117.171   192.168.100.43        6    153   
1   1.0   1712228535.151954  192.168.100.43   52.168.117.171        6     89   
2   1.0  1712228535.2723904  52.168.117.171   192.168.100.43        6     56   
3   1.0  1712228535.6919348  192.168.100.33  192.168.100.255       17     63   
4   1.0  1712228536.1014876  192.168.100.33  192.168.100.255       17     63   
..  ...                 ...             ...              ...      ...    ...   
87  1.0  1712228547.7763643   192.168.100.2      224.0.0.251       17    344   
88  1.0  1712228548.3907464   192.168.100.2      224.0.0.251       17    335   
89  1.0  1712228548.3907464  192.168.100.16  255.255.255.255       17    214   
90  1.0  1712228549.8233798   192.168.100.2      224.0.0.251       17     86   
91  1.0  1712228549.8243816   192.168.100.2      224.0.0.251       17    344   

    Info  
0    1.0  
1    1.0  
2    1

c:\Users\52333\Documents\Cybersecurity\clean_paquetes.py:56: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Source'] = df['Source'].str.replace('.', '')
c:\Users\52333\Documents\Cybersecurity\clean_paquetes.py:57: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Destination'] = df['Destination'].str.replace('.', '')


el shape del dataset torch.Size([66, 25, 3, 1])
66 25 3


In [40]:
from torch import nn


def predict(model, dataset):
  predictions, losses = [], []
  criterion = nn.L1Loss(reduction='sum').to(device)
  with torch.no_grad():
    model = model.eval()
    for seq_true in dataset:
      seq_true=seq_true.reshape((25,1,3))
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      seq_pred=seq_pred.reshape((25,1,3))
      loss = criterion(seq_pred, seq_true)

      predictions.append(seq_pred.cpu().numpy().flatten())
      losses.append(loss.item())
  return predictions, losses

In [45]:
#cargamos los pesos del modelo lstm autoencoder
from torch import nn
model_path = '25model.pth'
#loaded_model = torch.load(model_path)
loaded_model = torch.load('25model.pth', map_location=torch.device('cpu'))

#convert to numpy
#train_dataset = np.ndarray(train_dataset)
print(train_dataset.shape)
predictions,losses= predict(loaded_model, train_dataset)



torch.Size([66, 25, 3, 1])


In [47]:
losses[0]

0.3367134630680084

In [38]:
losses[0]

[array([ 1.44350544e-01,  4.63852659e-04,  9.74936843e-01,  3.21421295e-01,
         9.70503449e-01,  8.05367678e-02,  6.67523623e-01,  5.56614250e-05,
         2.59404033e-02, -1.12444162e-03,  1.49443746e-04,  2.33361945e-02,
         1.64090216e-01,  6.49347901e-04,  3.33419859e-01,  1.67407334e-01,
         3.42261046e-05,  4.20585275e-05,  4.07437563e-01,  9.75080669e-01,
         4.99750674e-03,  1.38833806e-01,  4.44171950e-04,  2.25209758e-01,
         1.88513219e-01,  4.09111381e-05,  5.53191081e-03,  7.82505870e-01,
         9.77302611e-01,  6.73077255e-02,  1.63951531e-01,  4.11354005e-04,
         2.01093286e-01,  8.95049632e-01,  9.70236182e-01,  4.00411040e-02,
         3.28457236e-01,  4.70411032e-05,  7.98366964e-05,  1.39332950e-01,
         3.39467078e-05,  4.61027026e-04,  6.52076006e-02,  9.62995946e-01,
         2.44186074e-03,  1.49850786e-01,  2.91913748e-04,  1.22447394e-01,
         1.64090216e-01,  6.49347901e-04,  3.33419859e-01,  2.57043213e-01,
         9.6

In [61]:
THRESHOLD=0.5421

correct = sum(l > THRESHOLD for l in losses)
print(f'anomaly predictions: {correct}/{len(train_dataset)}')


anomaly predictions: 2/66
